# Загрузка Pandas и очистка данных

In [32]:
RANDOM_SEED = 42

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import minmax_scale
import ast

In [29]:
def listfstr(s):
    """Функция для преобразования строки, представляющей собой список, в тип "список""""
    try:
        res = ast.literal_eval(s)
    except:
        res = []
    finally:
        return res
    
def find_item(cell):
    """Функция для проверки вхождения значения в ячейку cell
    
    """
    return 1 if item in cell else 0

def price_lable_encoding(p):
    if p == '$':
        return 1
    elif p== '$$ - $$$':
        return 2
    elif p== '$$$$':
        return 3
    else:
        return 0

SyntaxError: EOL while scanning string literal (<ipython-input-29-1ed2e7018772>, line 2)

In [40]:
df = pd.read_csv('main_task.xls')
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [31]:
### признак отсутствия данных по всем неполным столбцам
df['Number_of_Reviews_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')
df['Price_Range_isNAN'] = pd.isna(df['Price Range']).astype('uint8')
df['Cuisine_Style_isNAN'] = pd.isna(df['Cuisine Style']).astype('uint8')


### Нормируем ранг ресторана в группировке по городам
df['Ranking_Norm'] = df.groupby('City').Ranking.transform(lambda x: minmax_scale(x.astype(float)))


### dummies вместо "Price Range"
df = pd.concat([df, pd.get_dummies(df['Price Range'])], axis=1)
# df['Price_encoded'] = df['Price Range'].apply(price_lable_encoding)


### dummies вместо "City"
df = pd.concat([df, pd.get_dummies(df['City'])], axis=1)


### dummies вместо "Cuisine Style"
df['Cuisine_Style_list'] = df['Cuisine Style'].apply(listfstr)

# перебираем все рестораны, составляя set всех кухонь
cuisines = set() 
for cuisine_list in df['Cuisine_Style_list']:  
    for с in cuisine_list:
        cuisines.add(с)

# формируем dummies, описывающий набор кухонь для каждого ресторана
for item in cuisines:
    df[item] = df['Cuisine_Style_list'].apply(find_item)

# доп. признак: количество разных стилей кухонь, указанных в ресторане.
df['Cuisine_style_cnt'] = df['Cuisine_Style_list'].apply(len)


### считаем разницу между первым и последним отзывом 
df["Reviews"] = df["Reviews"].str.replace("nan", "' '")
df["Reviews"].fillna("[[], []]", inplace=True)

# вычленяем из текста даты
df["Reviews_list"] = df["Reviews"].apply(listfstr)
df["Reviews Dates"] = df["Reviews_list"].apply(lambda x: [datetime.strptime(d, "%m/%d/%Y") for d in x[1]])

# рассчитываем, как давно был оставлен последний отзыв
df["Review_Days_Ago"] = df["Reviews Dates"].apply(
 lambda x: (datetime.today() - max(x)).days if len(x) >= 1 else -1
)

### заполняем пропуски в "Number of Reviews"
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

In [32]:
df = df.drop(['City','Cuisine Style','Cuisine_Style_list','Price Range',
              'Reviews','Reviews Dates','Reviews_list','URL_TA','ID_TA', 'Ranking'], axis = 1)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 168 entries, Restaurant_id to Review_Days_Ago
dtypes: float64(3), int64(127), object(1), uint8(37)
memory usage: 41.4+ MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [34]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [35]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [36]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [37]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [38]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [39]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20635299999999998
